# hdpws and cdasws Example Jupyter Notebook
![SPASE inside](https://spase-group.org/assets/images/spase-inside.png)
This [Jupyter notebook](https://jupyter.org/) demonstrates using the [hdpws](https://pypi.org/project/hdpws/) Python package to access [Space Physics Archive Search and Extract](https://spase-group.org/) (SPASE) metadata documents from the [Heliophysics Data Portal](https://heliophysicsdata.gsfc.nasa.gov/) (HDP).  Additionally, it utilizes [cdasws](https://pypi.org/project/cdasws/) to retrieve the data described by the SPASE metadata.  It assumes some familarity with the [SPASE data model](https://spase-group.org/data/index.html).  This notebook contains the following sections:
1. [Prerequisites](#Prerequisites)
2. [Setup](#Setup)
3. [Get MeasurementTypes](#Get-MeasurementTypes)
4. [Get ObservedRegions](#Get-ObservedRegions)
5. [Get ObservatoryIDs](#Get-ObservatoryIDs)
6. [Get NumericalData](#Get-NumericalData)
7. [Additional Documentation](#Additional-Documentation)

## Prerequisites
Install the prerequisite software from [Python Package Index](https://pypi.org/project/hdpws/) (PyPI) software repository.
1. pip install hdpws
2. pip install xarray
3. pip install cdflib
4. pip install cdasws

## Setup
Execute some preliminary code that is necessary before the code that follows.

In [1]:
from hdpws.hdpws import HdpWs
from hdpws import NAMESPACES as NS
from hdpws.resourcetype import ResourceType as rt
from hdpws.spase import AccessURL

from cdasws import CdasWs
from cdasws.datarepresentation import DataRepresentation as dr

from IPython.core.display import HTML

# disable warnings about vspo-dev certificate
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

hdp = HdpWs(endpoint='http://ec2-44-212-210-55.compute-1.amazonaws.com/WS/hdp/1/',
           disable_ssl_certificate_validation=True)
cdas = CdasWs()

## Get MeasurementTypes
The following code demonstrates how to get the list of available /Spase/MeasurementType values.

In [2]:
result = hdp.get_measurement_types()
print('HDP MeasurementTypes:')
for value in result['MeasurementType']:
    print(f'    {value}')


HDP MeasurementTypes:
    ActivityIndex
    Dopplergram
    Dust
    ElectricField
    EnergeticParticles
    Ephemeris
    ImageIntensity
    InstrumentStatus
    IonComposition
    Irradiance
    MagneticField
    Magnetogram
    NeutralAtomImages
    NeutralGas
    Profile
    Radiance
    Spectrum
    SPICE
    ThermalPlasma
    Waves
    Waves.Active
    Waves.Passive


## Get ObservedRegions
The following code demonstrates how to get the list of available /Spase/ObservedRegion values.

In [3]:
result = hdp.get_observed_regions()
observed_regions = result['ObservedRegion']
print(f'{len(observed_regions)} HDP Observed Regions:')
for value in observed_regions[0:9]:
    print(f'    {value}')
print('    ...')

122 HDP Observed Regions:
    Asteroid
    Comet
    Earth
    Earth.Magnetosheath
    Earth.Magnetosphere
    Earth.Magnetosphere.Magnetotail
    Earth.Magnetosphere.Main
    Earth.Magnetosphere.Plasmasphere
    Earth.Magnetosphere.Polar
    ...


## Get ObservatoryIDs
The following code demonstrates how to get the list of available /Spase/Observatory/ResourceID values.

In [4]:
result = hdp.get_observatory_ids()
observatory_ids = result['ObservatoryID']
print(f'{len(observatory_ids)} HDP ObservatoryIDs:')
for value in observatory_ids[0:9]:
    print(f'    {value}')
print('    ...')

2003 HDP ObservatoryIDs:
    spase://SMWG/Observatory/JSPO
    spase://SMWG/Observatory/MEASURE/Darksky.Observatory
    spase://SMWG/Observatory/THEMIS/Ground/UCLA-EPO/BMLS
    spase://SMWG/Observatory/THEMIS/Ground/UCLA-GBO/GBAY
    spase://SMWG/Observatory/THEMIS/E
    spase://SMWG/Observatory/Ground/Karmoy
    spase://SMWG/Observatory/Ground/Val.Joyeux
    spase://SMWG/Observatory/Ground/Pangnirtung
    spase://SMWG/Observatory/Ground/Kevo
    ...


## Get NumericalData
The following code demonstrates how to find SPASE NumericalData documents matching the specified search criteria.   Additionally, it utilizes [cdasws](https://pypi.org/project/cdasws/) to retrieve the data described by the SPASE metadata when the data is available from CDAWeb.

In [5]:
query = {
    'InstrumentID': 'spase://SMWG/Instrument/ACE/MAG',
    'Cadence': '=PT4M',
    'ObservedRegion': 'Heliosphere.NearEarth',
    'MeasurementType': 'MagneticField',
    'AccessRights': 'Open',
    'Style': 'WebService'
}       
types = [rt.NUMERICAL_DATA]
time_range = ['2022-01-01', '2022-01-02']
result = hdp.get_spase_data(types, query, time_range)
if result['HttpStatus'] == 200:
    for spase in result['Result'].findall('.//Spase', namespaces=NS):
        id = spase.find('.//ResourceID', namespaces=NS)
        name = spase.find('.//Name', namespaces=NS)
        description = spase.find('.//Description', namespaces=NS)
        #print('Name: ', name.text)
        print('ResourceID: ', id.text)
        print('Description: ', description.text[:60], '...')
        display(HTML('<a href="' + hdp.get_spase_url(id.text) +
                     '" target="_blank">' + 
                    'HTML representation of SPASE</a>'))
        ws_access_url_element = \
            spase.find('.//AccessURL[Style="WebService"]', namespaces=NS)
        if ws_access_url_element is not None:
            ws_access_url = AccessURL(ws_access_url_element)
            ws_name = ws_access_url.name
            ws_product_key = ws_access_url.product_key[0]
            if "CDAWeb" in ws_name:
                display(HTML('<a href="' + ws_access_url.url + 
                             '" target="_blank">' + 
                             'Example data access code</a>'))
                print('Retrieving data...')
                example_interval = cdas.get_example_time_interval(ws_product_key)
                var_names = cdas.get_variable_names(ws_product_key)
                data = cdas.get_data(ws_product_key, var_names,
                                     example_interval,
                                     dataRepresentation = dr.XARRAY)[1]
                print(data)

ResourceID:  spase://NASA/NumericalData/ACE/MAG/L2/PT4M
Description:  This Data Product contains Measurements from the ACE Magneti ...


Retrieving data...
<xarray.Dataset>
Dimensions:     (Epoch: 31, cartesian: 3, dim0: 3)
Coordinates:
  * Epoch       (Epoch) datetime64[ns] 2023-01-16T21:56:00 ... 2023-01-16T23:...
  * cartesian   (cartesian) <U11 'x_component' 'y_component' 'z_component'
    metavar0    (cartesian) <U6 'Bx GSE' 'By GSE' 'Bz GSE'
    metavar2    (cartesian) <U9 'ACE X-GSE' 'ACE Y-GSE' 'ACE Z-GSE'
    metavar3    (cartesian) <U9 'ACE X-GSM' 'ACE Y-GSM' 'ACE Z-GSM'
Dimensions without coordinates: dim0
Data variables:
    Magnitude   (Epoch) float32 3.729 4.119 3.994 4.306 ... 5.574 5.541 5.576
    BGSEc       (Epoch, cartesian) float32 -0.257 -2.037 3.041 ... 1.307 1.958
    BGSM        (Epoch, dim0) float32 -0.257 -2.971 2.138 ... -5.05 0.525 2.295
    SC_pos_GSE  (Epoch, cartesian) float32 1.47e+06 2.422e+05 ... -4.691e+04
    SC_pos_GSM  (Epoch, cartesian) float32 1.47e+06 2.432e+05 ... 4.233e+04
    metavar1    (dim0) <U8 'Bx (GSM)' 'By (GSM)' 'Bz (GSM)'
Attributes: (12/29)
    TITLE:                

## Additional Documentation
View the [hdpws](https://heliophysicsdata.gsfc.nasa.gov/WebServices/py/hdpws/) and [cdasws](https://cdaweb.gsfc.nasa.gov/WebServices/REST/py/cdasws/) API documentation for a description of additional features.